<a href="https://colab.research.google.com/github/StacyChebet/Dyslexicon/blob/master/dyslexicon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dyslexicon CNN Model**

**Dyslexicon is a CNN model that predicts the likelihood that a child is suffering from Dyslexia.**

In [1]:
! pip install -q kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download drizasazanitaisa/dyslexia-handwriting-dataset

 93% 89.0M/96.2M [00:00<00:00, 166MB/s]
100% 96.2M/96.2M [00:00<00:00, 149MB/s]


In [8]:
! unzip dyslexia-handwriting-dataset.zip

Archive:  dyslexia-handwriting-dataset.zip
replace Dataset Dyslexia_Password WanAsy321.rar? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [16]:
#Importing modules
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import PIL
import PIL.Image
import tensorflow_datasets as tfds
import pathlib
import cv2


In [10]:
print (tf.__version__)

2.9.2


In [11]:
!pip install unrar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow==2.9.2

In [ ]:
! unrar x -Y /content/dyslexiaHandwriting.rar

In [17]:
#Initializing the CNN
classifier = Sequential()

In [18]:
#Step 1 - Convolution
classifier.add(Conv2D(512, (3,3), input_shape = (32,32,3), activation = 'relu'))
classifier.add(Conv2D(512, (3,3), activation = 'relu'))

In [19]:
#Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [20]:
#Adding a second convolutional layer
classifier.add(Conv2D(128, (3,3), activation = 'relu'))
classifier.add(Conv2D(128, (3,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [21]:
#Step 3 - Flattening
classifier.add(Flatten())

In [22]:
#Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))


In [23]:
#Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
#Getting the model's summary
classifier.summary()

In [25]:
#Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

test_data_gen = ImageDataGenerator(rescale = 1./255)

training_set = train_data_gen.flow_from_directory(r'/content/Gambo/Train',
                                                  target_size = (32,32),
                                                  batch_size = 32,
                                                  class_mode = 'categorical')

Found 151649 images belonging to 3 classes.


In [26]:
test_set = test_data_gen.flow_from_directory(r'/content/Gambo/Test',
                                             target_size = (32,32),
                                             batch_size = 32,
                                             class_mode = 'categorical')

Found 56723 images belonging to 3 classes.


In [ ]:
classifier.fit(training_set,
               steps_per_epoch = 2000,
               epochs = 20,
               validation_data = test_set,
               validation_steps = 2000)

Epoch 1/20
 924/2000 [============>.................] - ETA: 1:56:09 - loss: 0.6534 - accuracy: 0.6994

In [ ]:
#Saving the model
classifier.save('trained_model')

In [ ]:
#Part 3 - Making new predictions
